In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
BASE_DIR = Path().absolute().parent.parent
sys.path.append(str(BASE_DIR))

In [3]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case3T
import numpy as np
import pandas as pd

In [ ]:
DIR = BASE_DIR/'scripts_res'/'Algorithms'
DIR

WindowsPath('d:/OneDrive/Current/2_Codes/BO_Related/ASBO/ASBO/scripts_res')

### ASBO

In [5]:
re_run_ASBO = True
re_run_save_ASBO = True
sol = 'ASBO'
case_name = 'Case3T'
runs_number = 100

In [6]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

obj_list = []
x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []
all_res_list = []

def func(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3T(x)
    cons = [c1,c2,c3,c4]
    return -obj,cons

nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)

if re_run_ASBO == True or re_run_save_ASBO == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        try:
            optimizer.maximize(init_points=100, n_iter=200,p_hyper=[[5, 2, 0.78, 0.95,1e-5]],acquisition_function=acquisition_function)
            print(optimizer.max)
            x_1_list.extend([optimizer.max['params']['x1']])
            x_2_list.extend([optimizer.max['params']['x2']])
            x_3_list.extend([optimizer.max['params']['x3']])
            x_4_list.extend([optimizer.max['params']['x4']])
            x_5_list.extend([optimizer.max['params']['x5']])  
            obj_list.extend([optimizer.max['target']])                                  
            con_1_list.extend([optimizer.max['constraint'][0]])
            con_2_list.extend([optimizer.max['constraint'][1]])
            con_3_list.extend([optimizer.max['constraint'][2]])
            con_4_list.extend([optimizer.max['constraint'][3]])
            all_res = [item['target'] for item in optimizer.res]
            all_fea = [item['allowed'] for item in optimizer.res]
            modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
            all_res_list.append(modified_res)            
        except Exception as e:
            print(e)
            obj_list.append(None)
            x_1_list.append(None)
            x_2_list.append(None)
            x_3_list.append(None)
            x_4_list.append(None)
            x_5_list.append(None)                        
            con_1_list.append(None)
            con_2_list.append(None)
            con_3_list.append(None)
            con_4_list.append(None) 
            modified_res = [-np.inf for _ in range(100+200)]
            all_res_list.append(modified_res)   

        df2 = pd.DataFrame(all_res_list, columns=[list(range(100+200))])               
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'con_4': con_4_list
        })
        if re_run_save_ASBO == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
            df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)


{'target': -6.678419461780873, 'params': {'x1': 2.0527836354692997, 'x2': 0.053264070707971005, 'x3': -3.998679927117077, 'x4': 3.0527558087201436, 'x5': -6.90362662545977}, 'constraint': array([1.92161124, 1.14055315, 1.01020561, 1.45613195])}
{'target': -2.2587733961932384, 'params': {'x1': 2.0615177374319495, 'x2': -2.050129239251082, 'x3': -0.9112602057448643, 'x4': -2.071441760080008, 'x5': 2.0823542614743022}, 'constraint': array([-0.49780558,  0.93551768,  0.01876627,  0.02996947])}
{'target': -6.581341365154717, 'params': {'x1': 7.994910233621434, 'x2': -5.0036740776721516, 'x3': 0.9983497857961334, 'x4': 4.005092436898134, 'x5': -4.9960542778511865}, 'constraint': array([-0.05070784,  0.98764613, -0.21264755,  1.4637446 ])}
{'target': 0.669590513553, 'params': {'x1': -0.9834820927188529, 'x2': -0.9815644952400481, 'x3': 2.968490294153601, 'x4': 2.006590726100356, 'x5': 1.0046622805621597}, 'constraint': array([ 1.8583747 , -0.8345768 ,  1.23534248,  0.78564985])}
{'target': -1

In [7]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 14


,obj,x_1,x_2,x_3,x_4,x_5,con_1,con_2,con_3,con_4
count,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000
mean,-5.550912,2.190889,1.567127,-0.523945,4.052930,0.517206,0.716738,0.281669,1.098151,0.695075
std,5.107125,3.657555,5.196632,3.789806,5.107535,4.048954,1.030299,1.038919,0.991146,0.490579
min,-18.274133,-4.047029,-7.015833,-7.998549,-2.967179,-6.977986,-0.953232,-0.999454,-0.380128,0.004984
25%,-8.276288,0.000418,-1.993824,-3.918813,0.030836,-2.754162,-0.249653,-0.779250,0.282109,0.178501
50%,-6.062201,1.182011,0.079193,-0.046543,2.525499,0.022221,0.781679,0.461837,1.018536,0.743369
75%,-1.721511,3.999933,4.948357,1.049867,7.012763,2.968685,1.770319,0.996510,1.765022,1.094469
max,4.483473,11.101903,14.036088,7.048851,16.071576,10.004263,1.955078,3.072895,2.999713,1.482362


### CBO

In [5]:
re_run_CBO = True
re_run_save_CBO = True
sol = 'CBO'
case_name = 'Case3T'
runs_number = 100

In [6]:
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

obj_list = []
x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []
all_res_list = []

def func_obj(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3T(x)
    cons = [c1,c2,c3,c4]
    # print(('cons, obj value=', cons, obj))
    return -obj

def func_cons(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3T(x)
    cons = [c1,c2,c3,c4]
    return cons

nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func_cons, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=1)

if re_run_CBO == True or re_run_save_CBO == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func_obj, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        try: 
            optimizer.maximize(init_points=100, n_iter=200,acquisition_function=acquisition_function)
            print(optimizer.max)
            x_1_list.extend([optimizer.max['params']['x1']])
            x_2_list.extend([optimizer.max['params']['x2']])
            x_3_list.extend([optimizer.max['params']['x3']])
            x_4_list.extend([optimizer.max['params']['x4']])
            x_5_list.extend([optimizer.max['params']['x5']])   
            obj_list.extend([optimizer.max['target']])                     
            con_1_list.extend([optimizer.max['constraint'][0]])
            con_2_list.extend([optimizer.max['constraint'][1]])
            con_3_list.extend([optimizer.max['constraint'][2]])
            con_4_list.extend([optimizer.max['constraint'][3]])
            all_res = [item['target'] for item in optimizer.res]
            all_fea = [item['allowed'] for item in optimizer.res]
            modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
            all_res_list.append(modified_res)  

        except Exception as e:
            print(e)
            obj_list.append(None)
            x_1_list.append(None)
            x_2_list.append(None)
            x_3_list.append(None)
            x_4_list.append(None)
            x_5_list.append(None)                        
            con_1_list.append(None)
            con_2_list.append(None)
            con_3_list.append(None)
            con_4_list.append(None)        
            modified_res = [-np.inf for _ in range(100+200)]
            all_res_list.append(modified_res)   
        df2 = pd.DataFrame(all_res_list, columns=[list(range(100+200))])                 
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,        
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'con_4': con_4_list
        })
        if re_run_save_CBO == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
            df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)

{'target': 0.9697029284995078, 'params': {'x1': 1.0134936955326663, 'x2': 0.886332302741131, 'x3': -1.0335850955648895, 'x4': -0.9983418861183083, 'x5': 0.15444592932881804}, 'constraint': array([0.68342857, 0.00386688, 2.57604045, 0.69079835])}
Data point [ 2.81796452  2.94656627  5.97207182 -2.85699674  1.17836625] is not unique. You can set "allow_duplicate_points=True" to avoid this error
{'target': -7.992512169610684, 'params': {'x1': -0.6050995533057159, 'x2': 0.325224734335215, 'x3': 1.8300020034885038, 'x4': 1.292510884544572, 'x5': -0.7538479338939688}, 'constraint': array([-0.64085479,  2.57518878,  2.49840949,  1.17795271])}
{'target': -5.334211885537663, 'params': {'x1': 1.1178396763510523, 'x2': -0.9014598315164208, 'x3': 1.1853274260062185, 'x4': 0.15365555283914434, 'x5': -0.46997084194941985}, 'constraint': array([0.40336814, 2.80983053, 2.79945941, 0.91315658])}
{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
Data point [ 2.9

In [7]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 51


,obj,x_1,x_2,x_3,x_4,x_5,con_1,con_2,con_3,con_4
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,-4.881060,0.155385,0.294437,0.417939,0.124925,0.159916,0.439841,0.625410,2.560287,0.721442
std,2.810909,0.894869,1.034374,1.056542,0.861739,1.050510,0.650486,1.112805,0.355770,0.429704
min,-9.195227,-1.130379,-1.964408,-1.738786,-1.414605,-1.941270,-0.775958,-0.995338,1.259545,0.053160
25%,-6.534886,-0.869925,-0.642108,-0.663963,-0.864475,-0.753848,-0.042595,-0.260550,2.422351,0.407008
50%,-5.781908,0.145934,0.272120,0.702747,0.286324,0.036930,0.408171,0.378628,2.633329,0.685147
75%,-3.651482,0.933458,1.024831,1.185327,0.852084,0.923648,0.957288,1.264816,2.831675,1.095472
max,2.464712,2.144818,2.552243,2.150462,1.889570,2.483306,1.807303,3.098033,2.986737,1.491662


### SCBO

In [5]:
re_run_SCBO = True
re_run_save_SCBO = True
sol = 'SCBO'
case_name = 'Case3T'
runs_number = 100

In [6]:
import gpytorch
import torch
import tr_bo
from torch.quasirandom import SobolEngine
from botorch.utils.transforms import unnormalize
from botorch.models.model_list_gp_regression import ModelListGP

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
iterations = []
max_values = []
all_res_list = []

def func(x):
    obj, c1, c2, c3, c4 = Case3T(x)
    return -obj,c1,c2, c3,c4

device = torch.device("cuda" if torch.cuda.is_available() else"cpu")
dtype = torch.double
dim = 5
batch_size = 1
n_init = 100 
bo_iter = 200
max_cholesky_size = float("inf")  # Always use Cholesky

lb = torch.tensor([-1.0, -1, -2.0, 0,], dtype=torch.float64)
ub =  torch.tensor([2.0, 4, 3.0, 1.5, ], dtype=torch.float64)
bounds = torch.tensor([[-5, -20,  -10, -12, -15], [12, 22, 10, 20, 18]])

def objective(x):
    x_numpy = x.numpy()
    res = func(x_numpy)
    # res = func(x)
    target = res[0]
    c1 = res[1]
    c2 = res[2]
    c3 = res[3]
    c4 = res[4]
    return target,c1,c2,c3,c4
def run_multiple_calculations(num_runs):
    for random_seed in range(num_runs):
        try:
            def eval_objective(x):
                return objective(unnormalize(x, bounds))
            # Initialize TuRBO state
            state = tr_bo.ScboState(dim, batch_size=batch_size)
            sobol = SobolEngine(dim, scramble=True, seed=1)

            # Get initial data
            # Must get initial values for both objective and constraints
            train_X = tr_bo.get_initial_points(dim, n_init,seed=random_seed)

            train_Y_tensor_list = []
            C1_tensor_list = []
            C2_tensor_list = []
            C3_tensor_list = []
            C4_tensor_list = []
            for x in train_X:
                result = eval_objective(x)
                combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                train_Y, C1, C2,C3,C4 = combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3], combined_tensor[4]
                train_Y_tensor_list.append(train_Y)
                C1_tensor_list.append(C1)
                C2_tensor_list.append(C2)
                C3_tensor_list.append(C3)
                C4_tensor_list.append(C4)
            
            train_Y = torch.stack(train_Y_tensor_list, dim=0)
            C1 = torch.stack(C1_tensor_list, dim=0)
            C2 = torch.stack(C2_tensor_list, dim=0)
            C3 = torch.stack(C3_tensor_list, dim=0)
            C4 = torch.stack(C4_tensor_list, dim=0)
            while not state.restart_triggered:  # Run until TuRBO converges
                # Fit GP models for objective and constraints
                model = tr_bo.get_fitted_model(train_X, train_Y,dim)
                c1_model = tr_bo.get_fitted_model(train_X, C1,dim)
                c2_model = tr_bo.get_fitted_model(train_X, C2,dim)
                c3_model = tr_bo.get_fitted_model(train_X, C3,dim)
                c4_model = tr_bo.get_fitted_model(train_X, C4,dim)
                # Generate a batch of candidates
                with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                    X_next = tr_bo.generate_batch(
                        state=state,
                        model=model,
                        X=train_X,
                        Y=train_Y,
                        batch_size=batch_size,
                        n_candidates=2000,
                        constraint_model=ModelListGP(c1_model, c2_model,c3_model,c4_model),
                        sobol=sobol,
                    )

                Y_next_tensor_list = []
                C1_next_tensor_list = []
                C2_next_tensor_list = []
                C3_next_tensor_list = []
                C4_next_tensor_list = []
                for x in X_next:
                    result = eval_objective(x)
                    combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                    Y_next, C1_next, C2_next, C3_next, C4_next,= combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3], combined_tensor[4]
                    Y_next_tensor_list.append(Y_next)
                    C1_next_tensor_list.append(C1_next)
                    C2_next_tensor_list.append(C2_next)
                    C3_next_tensor_list.append(C3_next)
                    C4_next_tensor_list.append(C4_next)
                Y_next = torch.stack(Y_next_tensor_list, dim=0)
                C1_next = torch.stack(C1_next_tensor_list, dim=0)
                C2_next = torch.stack(C2_next_tensor_list, dim=0)
                C3_next = torch.stack(C3_next_tensor_list, dim=0)
                C4_next = torch.stack(C4_next_tensor_list, dim=0)

                C_next = torch.cat([C1_next, C2_next,C3_next, C4_next,], dim=-1)
                state = tr_bo.update_state(state, Y_next, C_next,ub,lb)
                train_X = torch.cat((train_X, X_next), dim=0)
                train_Y = torch.cat((train_Y, Y_next), dim=0)
                C1 = torch.cat((C1, C1_next), dim=0)
                C2 = torch.cat((C2, C2_next), dim=0)
                C3 = torch.cat((C3, C3_next), dim=0)
                C4 = torch.cat((C4, C4_next), dim=0)
                C = torch.cat([C1, C2, C3, C4], dim=-1)
                bool_tensor = (C >= lb.repeat(len(C), 1)) & (C <= ub.repeat(len(C), 1))
                bool_tensor = torch.all(bool_tensor, dim=-1)
                Valid_Y = train_Y[bool_tensor]
                Valid_X = train_X[bool_tensor]

                if len(Valid_Y)!=0:
                    max_index = torch.argmax(Valid_Y)  
                else:
                    print(f"{len(train_X)}) No feasible point yet! TR length: {state.length:.2e}")  

                if len(train_X)==n_init+bo_iter:
                    modified_Y = torch.where(bool_tensor, train_Y.flatten(), torch.tensor(float('-inf')))
                    all_res_list.append(modified_Y.tolist())
                    df2 = pd.DataFrame(all_res_list, columns=[list(range(n_init+bo_iter))])     

                    if len(Valid_Y)!=0:
                        max_value = Valid_Y[max_index].item()
                        c_x = Valid_X[max_index]
                        print("obj= ",max_value, ' x = ', c_x)
                        max_values.append(max_value)
                        x_1_list.append(c_x[0].item())
                        x_2_list.append(c_x[1].item())
                        x_3_list.append(c_x[2].item())
                        x_4_list.append(c_x[3].item())
                        x_5_list.append(c_x[4].item())      
                    else:
                        print("No feasible solutions")
                        max_values.append(None)
                        x_1_list.append(None)
                        x_2_list.append(None)
                        x_3_list.append(None)
                        x_4_list.append(None)
                        x_5_list.append(None)                                                           
                    iterations.append(len(train_X))
                    df = pd.DataFrame({
                    'obj': max_values,
                    'x_1': x_1_list,
                    'x_2': x_2_list,
                    'x_3': x_3_list,
                    'x_4': x_4_list,
                    'x_5': x_5_list,                       
                    'iterations': iterations
                    })
                    if re_run_save_SCBO == True:
                        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
                        df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)
                    break

        except Exception as e:
            fixed_length_Y = torch.full((n_init+bo_iter,), float('-inf'))           
            modified_Y = torch.where(bool_tensor, train_Y.flatten(), torch.tensor(float('-inf')))
            fixed_length_Y[:modified_Y.size(0)] = modified_Y
            all_res_list.append(fixed_length_Y.tolist())
            df2 = pd.DataFrame(all_res_list, columns=[list(range(n_init+bo_iter))])
                        
            if len(train_X) == n_init + bo_iter:
                print(e)
            else:
                print(e, "Early Stop")
                if len(Valid_Y)!=0:
                    max_value = Valid_Y[max_index].item()
                    c_x = Valid_X[max_index]
                    print("obj= ",max_value, ' x = ', c_x)
                    max_values.append(max_value)
                    x_1_list.append(c_x[0].item())
                    x_2_list.append(c_x[1].item())
                    x_3_list.append(c_x[2].item())
                    x_4_list.append(c_x[3].item())
                    x_5_list.append(c_x[4].item())                        
                else:
                    print("obj= ", None, ' x = ', None)
                    max_values.append(None)
                    x_1_list.append(None)
                    x_2_list.append(None)
                    x_3_list.append(None)
                    x_4_list.append(None)
                    x_5_list.append(None)                
            iterations.append(len(train_X))
            df = pd.DataFrame({
            'obj': max_values,
            'x_1': x_1_list,
            'x_2': x_2_list,
            'x_3': x_3_list,
            'x_4': x_4_list,
            'x_5': x_5_list,               
            'iterations': iterations
            })
            if re_run_save_SCBO == True:
                df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
                df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)                

run_multiple_calculations(num_runs=runs_number)

101) No feasible point yet! TR length: 8.00e-01
102) No feasible point yet! TR length: 8.00e-01
103) No feasible point yet! TR length: 8.00e-01
104) No feasible point yet! TR length: 8.00e-01
105) No feasible point yet! TR length: 8.00e-01
106) No feasible point yet! TR length: 8.00e-01
107) No feasible point yet! TR length: 8.00e-01
108) No feasible point yet! TR length: 8.00e-01
109) No feasible point yet! TR length: 8.00e-01
110) No feasible point yet! TR length: 8.00e-01
111) No feasible point yet! TR length: 8.00e-01
112) No feasible point yet! TR length: 8.00e-01
113) No feasible point yet! TR length: 8.00e-01
114) No feasible point yet! TR length: 8.00e-01
115) No feasible point yet! TR length: 6.67e-01
116) No feasible point yet! TR length: 6.67e-01
117) No feasible point yet! TR length: 6.67e-01
118) No feasible point yet! TR length: 6.67e-01
119) No feasible point yet! TR length: 6.67e-01
120) No feasible point yet! TR length: 6.67e-01
121) No feasible point yet! TR length: 6

In [7]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 100


,obj,x_1,x_2,x_3,x_4,x_5,iterations
count,0.0,0.0,0.0,0.0,0.0,0.0,100.000000
mean,NaN,NaN,NaN,NaN,NaN,NaN,185.080000
std,NaN,NaN,NaN,NaN,NaN,NaN,12.551051
min,NaN,NaN,NaN,NaN,NaN,NaN,155.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,177.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,185.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,193.000000
max,NaN,NaN,NaN,NaN,NaN,NaN,219.000000


In [ ]:
**

### trust-constr

In [5]:
re_run_tc = True
re_run_save_tc = True
sol = 'trust-constr'
case_name = 'Case3T'
runs_number = 100

In [6]:
def func(x):
    obj, c1, c2, c3, c4 = Case3T(x)
    cons = [c1,c2,c3,c4]
    return obj,c1,c2,c3,c4

initial_x1 = []
initial_x2 = []
initial_x3 = []
initial_x4 = []
initial_x5 = []

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
obj_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []

nfev_r = []
success_r = []
message_r = []
def recyc(x):
    nc_lb = np.array([-1,-1,-2,0,])
    nc_ub = np.array([2,  4, 3,1.5])
    cons = NonlinearConstraint(lambda x: func(x)[1:5], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x[0]
    lb = [-5, -20,-10,-12,-15] 
    ub = [12,22,10,20,18] 
    bd = Bounds(lb, ub, True) 
    try:
        result = minimize(objective, x0,  method='trust-constr', bounds=bd,constraints=cons)
        print('obj = ', result.fun, ', x = ', result.x)
        obj_list.append(result.fun)
        x_1_list.append(result.x[0])
        x_2_list.append(result.x[1])
        x_3_list.append(result.x[2])
        x_4_list.append(result.x[3])
        x_5_list.append(result.x[4])        
        con_1_list.append(result.constr[0][0])
        con_2_list.append(result.constr[0][1])
        con_3_list.append(result.constr[0][2])    
        con_4_list.append(result.constr[0][3])    
        initial_x1.append(x[0][0])  
        initial_x2.append(x[0][1])  
        initial_x3.append(x[0][2])  
        initial_x4.append(x[0][3])  
        initial_x5.append(x[0][4])      
        nfev_r.append(result.nfev)
        success_r.append(str(result.success))
        message_r.append(result.message)

    except Exception as e:
        print(e)
        obj_list.append(None)
        x_1_list.append(None)
        x_2_list.append(None)
        x_3_list.append(None)
        x_4_list.append(None)
        x_5_list.append(None)        
        con_1_list.append(None)
        con_2_list.append(None)
        con_3_list.append(None)    
        con_4_list.append(None)    
        initial_x1.append(x[0][0])  
        initial_x2.append(x[0][1])  
        initial_x3.append(x[0][2])  
        initial_x4.append(x[0][3])  
        initial_x5.append(x[0][4])      
        nfev_r.append(None)
        success_r.append(None)
        message_r.append(e)

    df = pd.DataFrame({ 
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'con_4': con_4_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'initial_x3': initial_x3,
        'initial_x4': initial_x4,  
        'initial_x5': initial_x5,                 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })    
    if re_run_save_tc == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    # return result.x

def initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18] ,a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run_tc == True or re_run_save_tc == True:
    x_random = initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)

obj =  4.98940447656938 , x =  [ 2.05997362 -5.97085192  5.00327996  2.02052913  1.00955289]
obj =  10.17604334390381 , x =  [ 7.10189315 -4.02895091 -0.88385339  2.94516658 -5.14200436]
obj =  11.4857097454937 , x =  [ 5.08967826  2.01366308 -5.99825201  5.03765689 -7.21300988]
obj =  15.597683202681065 , x =  [ 0.14290838 12.79022169 -0.08020442 11.14595052 -0.03172594]
obj =  9.319310923091866 , x =  [ 6.9826923  -5.11942383  3.01619349  5.07230434 -4.08947133]
obj =  13.321913838334417 , x =  [ 6.13806586 -5.01678407  1.23490657  2.90870227 -4.20344618]


e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


obj =  6.596211504984598 , x =  [ 4.99931198 -7.99808312  4.99931198  2.99971275  2.99971275]
obj =  8.233632198968742 , x =  [-3.20737000e+00  5.98815332e+00 -1.89033574e+00 -7.59257490e-02
  9.11910389e-04]
obj =  7.466241018777582 , x =  [11.9590864   0.01921143 -2.99948136  0.03397418  0.02099906]
obj =  10.975701303281056 , x =  [ 0.4054822  -2.94194231  1.99696908  3.95988252  4.97186502]
obj =  19.049317180023273 , x =  [10.08013015 -5.60273961  5.16275549  7.27201643 -1.72401754]
`x0` violates bound constraints.
obj =  14.553385446048004 , x =  [ 5.92754902  6.57631315 -5.1156165   0.11980704  0.02063236]
`x0` violates bound constraints.
obj =  11.051638638062654 , x =  [ 4.94890711 -7.01234133 -7.05183721 -0.89235948  8.87822591]
obj =  12.877893441815228 , x =  [ 9.9546729  -0.81064709 -2.9424552   8.95289592 -7.09391936]


e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


obj =  15.6186593212051 , x =  [11.60671635 -3.87562257 -2.96610555  1.99061935  5.03358149]
obj =  8.23682563914722 , x =  [ 7.99968824  1.00029174  4.00003312 11.99934328 -2.99936348]
`x0` violates bound constraints.
`x0` violates bound constraints.
obj =  13.234508475296819 , x =  [ 4.94522685 -7.94160809  1.9429522  15.03528177  9.82600949]


...

obj =  16.565976866799538 , x =  [ 8.94735891 -5.37987821  0.78795368 -0.69796151  1.95867821]
`x0` violates bound constraints.
obj =  11.75528026749782 , x =  [ 8.04491372  3.97945044 -6.99783616 -2.76931794  2.07630259]
obj =  15.930343264300324 , x =  [10.84937624 12.0141133  -6.22175322  7.11412208 -1.91482228]
obj =  9.910290675191069 , x =  [ 5.99216847e+00  5.58037516e-03  7.00119960e+00  1.29004643e+01
 -9.40113664e-01]
obj =  18.569274288524205 , x =  [10.24287257  4.24005414 -7.00990511 -2.61998749  1.26333301]
obj =  7.662279333386602 , x =  [10.03105155 -4.02871996 -0.97273786  3.9813269  -5.03571142]
`x0` violates bound constraints.
obj =  13.81601361715006 , x =  [ 8.01514516  1.05160924 -9.94231529 -4.2729532   5.97316788]
obj =  18.396005766043505 , x =  [ 1.15643424  7.1917515   5.07270306 15.34386386  3.10971145]
`x0` violates bound constraints.
`x0` violates bound constraints.
obj =  16.89217141996293 , x =  [ 6.15118245 -5.83660536  4.10288904  4.39438355 -3.7289532

e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


obj =  16.386136391105037 , x =  [-0.68335293 -1.88983459 -6.9994964  15.04963323 11.99137618]
obj =  7.568646319579866 , x =  [ 2.99435904 10.99143848 -6.99908773  1.0071198   1.00278726]
obj =  20.173582662970105 , x =  [-1.23856207 17.0596962  -6.73452463  4.31345958  0.2851804 ]
obj =  7.709038329416746 , x =  [ 3.98423991 -7.1004665   5.99850215  4.01669898 -2.04045753]
`x0` violates bound constraints.
obj =  18.938618046969836 , x =  [ 2.61886009  8.4335518  -8.98586738 -2.94305297  3.61183668]
obj =  12.125048669379666 , x =  [ 6.95519986 -3.7280953  -0.93152123  2.94344021 -5.10356823]
`x0` violates bound constraints.
obj =  15.52196705968747 , x =  [ 0.02917884  6.69671207  5.92431162 14.09126202  0.95342208]
obj =  12.482064880712162 , x =  [-3.58811357  4.0091565  -4.00325678  2.00728495  5.9758173 ]
obj =  10.90235045784495 , x =  [ 8.01134979 -4.95255773  8.99965651 14.90602624  4.0503266 ]
obj =  15.047247363239704 , x =  [ 4.91065356  5.9372536   4.00949836 13.75234446  

e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


obj =  5.852109753304344 , x =  [-1.0124622   0.64462929  1.90420818  2.02454947 -1.95800147]
obj =  7.809917669638736 , x =  [ 2.04123811 -5.97210821  7.99889352  8.01470024  2.99535842]
`x0` violates bound constraints.
obj =  18.795298940474357 , x =  [ 2.96416231 -6.99245139 -6.9742142  14.68340323 12.69522436]
obj =  7.0967437158704065 , x =  [ 3.99933239 -4.99916866  4.99917455  7.99870836 -4.9991698 ]
`x0` violates bound constraints.
obj =  13.285767004702926 , x =  [ 2.98860759 -1.35319059 -3.08891357  3.03769769 -6.80630922]
`x0` violates bound constraints.
obj =  12.481108440085885 , x =  [-2.3643618   3.13294227  0.09008787  4.9541028  -4.14469709]
obj =  15.202730731935059 , x =  [ 0.04308975 13.68855224 -4.06960575  6.090524    1.00198348]
obj =  16.933761485608002 , x =  [ 7.25291968 -4.52159315  0.37188255  2.19476169 -3.8399562 ]
obj =  14.992006317579369 , x =  [ 4.11517553 -7.8476739   6.07741792  2.29309267  2.14138266]
obj =  16.456385105719775 , x =  [-1.00003771 12

In [7]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df_cleaned = df.dropna(subset=['obj'])
df_cleaned.describe()

NaN count in column 'obj': 22


,obj,x_1,x_2,x_3,x_4,x_5,con_1,con_2,con_3,con_4,initial_x1,initial_x2,initial_x3,initial_x4,initial_x5,nfev
count,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000
mean,12.533434,4.362480,0.556385,-0.327276,5.510182,1.045695,0.795196,0.486254,0.527546,0.903011,3.315640,-1.265145,0.902933,4.045352,1.614519,2239.076923
std,4.400238,3.963302,7.302929,5.320168,5.613745,4.660380,1.384945,1.499305,0.690373,0.700981,4.378657,11.908263,5.541276,8.996431,9.208462,2073.933077
min,2.250873,-3.695486,-7.998083,-9.942315,-4.272953,-7.213010,-1.000000,-1.000000,-0.233799,-0.186871,-4.528347,-19.569498,-9.739653,-11.850767,-14.889930,312.000000
25%,8.386381,1.581882,-5.547024,-5.004118,1.140022,-2.481038,-0.997273,-0.745941,0.004607,0.089408,0.058138,-11.392734,-3.253120,-2.822096,-5.915186,991.500000
50%,12.679979,4.363492,-1.941782,-0.369309,4.217421,1.002385,1.978458,0.076486,0.288858,1.139050,3.794253,-1.783072,1.079466,3.654909,1.636018,1275.000000
75%,15.852422,7.775328,6.429273,4.079541,9.018845,3.486305,2.000000,1.103798,0.995621,1.500000,6.116922,10.455565,5.473187,10.663278,8.360708,2976.000000
max,21.048675,11.959086,17.059696,9.949520,17.890498,12.695224,2.042818,3.930601,2.998161,2.193548,11.397368,20.378388,9.777242,19.838408,17.950608,7572.000000


### slsqp

In [8]:
re_run_slsqp = True
re_run_save_slsqp = True
sol = 'slsqp'
case_name = 'Case3T'
runs_number = 100

In [ ]:
initial_x1 = []
initial_x2 = []
initial_x3 = []
initial_x4 = []
initial_x5 = []

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
obj_list = []

nfev_r = []
success_r = []
message_r = []

def func(x):
    obj, c1, c2, c3, c4 = Case3T(x)
    cons = [c1,c2,c3,c4]
    return obj,c1,c2,c3,c4

initial_x = []
fun_r = []
nfev_r = []
success_r = []
message_r = []
def recyc(x):
    nc_lb = np.array([-1,-1,-2,0,])
    nc_ub = np.array([2,  4, 3,1.5])
    cons = NonlinearConstraint(lambda x: func(x)[1:5], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x[0]
    lb = [-5, -20,-10,-12,-15] 
    ub = [12,22,10,20,18] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='slsqp', bounds=bd,constraints=cons)
    print('obj = ', result.fun, ', x = ', result.x)
    obj_list.append(result.fun)
    x_1_list.append(result.x[0])
    x_2_list.append(result.x[1])
    x_3_list.append(result.x[2])
    x_4_list.append(result.x[3])
    x_5_list.append(result.x[4])        

    initial_x1.append(x[0][0])  
    initial_x2.append(x[0][1])  
    initial_x3.append(x[0][2])  
    initial_x4.append(x[0][3])  
    initial_x5.append(x[0][4])      
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    df = pd.DataFrame({ 
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'initial_x3': initial_x3,
        'initial_x4': initial_x4,  
        'initial_x5': initial_x5,                 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })    
    if re_run_save_slsqp == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    return result.x

def initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18] ,a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run_slsqp == True or re_run_save_slsqp == True:
    x_random = initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)


obj =  13.618727088322887 , x =  [11.08574118 -4.039577   -0.92036074  9.94513399  7.19627677]
obj =  6.413194136372351 , x =  [ 2.93618858 -5.06139898  3.0956695   2.96711247 -4.0329463 ]


e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `fin

obj =  6.99684350966999 , x =  [ 5.99901919  1.99970759 -5.99895963  4.99916791 -6.99878965]
obj =  8.814908709439775 , x =  [ 1.0007331   7.99242396  4.00204789 13.97052669  1.02382998]
obj =  7.165182829659258 , x =  [ 8.00072982 -7.00608754  5.00203996  4.00408133 -2.00118694]
obj =  -0.021376044126256044 , x =  [ 4.00803461 -1.01187648 -0.98660545 -0.99173323 -0.99104394]
obj =  11.94994997489771 , x =  [ 6.07801245 -7.08207211  6.21058953  4.9881746   1.03841155]
obj =  9.207769231569761 , x =  [-2.98890664 15.00353056 -9.99955085  0.97890617 -4.01146847]
obj =  7.301643858886965 , x =  [ 7.99909153 -7.99851081  4.99957166  2.99982327  2.00000149]
obj =  13.289620208994966 , x =  [ 5.98154105 -7.67232083  5.99733337  3.0366227   0.03679809]
obj =  5.925397450912449 , x =  [ 6.00502029 -3.00600679 -0.9949345   4.99547828 -6.0061098 ]
obj =  12.817772189953056 , x =  [ 7.05963124 -7.98017543 -8.96383532 15.95948712 13.14805964]
obj =  11.992448940325884 , x =  [-4.16204865  9.966514

e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `fin

obj =  10.774049220035549 , x =  [-1.94986625 12.01142699 -9.99912416 -3.12764729  2.95104284]
obj =  8.78490843434667 , x =  [ 6.00447264 -7.03084038  6.00050915 11.00988081  6.00790216]
obj =  14.904545207954046 , x =  [ 1.14600134 -4.25293369  8.86786864  8.10004479 -0.9543073 ]
obj =  15.044430695369226 , x =  [-0.7161535  15.10956664 -5.01645203  6.02340008  0.99427046]
obj =  10.722930012849531 , x =  [ 4.10512004  8.98310085 -3.93095034  7.94460723 -4.1011163 ]
obj =  9.961221328740004 , x =  [ 1.65109089e-04  5.99262368e+00 -3.99501881e+00 -7.08182854e-01
 -8.76845098e-01]
obj =  8.97722096641224 , x =  [ 8.97229156  3.02511635 -9.02072613  5.01387654 -7.95339488]
obj =  0.06864775016503133 , x =  [-1.03978264 -1.02802188  1.99699254  1.98101437  2.99434695]
obj =  15.708847994366094 , x =  [-0.29636092 14.95255906 -6.12129889  4.06654439 -0.95547353]


...

e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `fin

obj =  11.792991808038103 , x =  [ 8.93240326 -7.8414324   7.99936191  9.0322835   2.98870868]
obj =  13.617465405806737 , x =  [10.70305461 -5.94258301  2.98342386  3.05733317  2.00604288]
obj =  13.498250228013942 , x =  [ 0.05962827  8.79674796  3.94794752 15.05671985  2.96263217]
obj =  8.858308534012679 , x =  [-4.1097518   8.95164043 -4.91466779 -1.04583413  0.99962948]
obj =  9.178820076941236 , x =  [ 8.00664417  8.99243495 -0.99485016 13.99601514  4.00526962]
obj =  10.01452353666662 , x =  [-0.92334851 15.03085819 -2.99984131 10.03262771 -2.02128984]
obj =  14.686776010568245 , x =  [ 4.06349735 11.30088651 -2.97428438 10.0047993   3.01997135]
obj =  11.565682195344099 , x =  [-3.13355486  2.84833573 -2.87735519  1.89400342  5.22473574]
obj =  10.516943591286028 , x =  [-0.96342068  1.02108798 -1.99675518 16.91114934 10.03137428]


e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `fin

obj =  14.100135111019657 , x =  [-3.63211081  6.00298403 -5.01442587  4.97570184  6.93531585]
obj =  13.92092842054405 , x =  [ 1.19602595e+01  1.42086914e+01 -6.94089670e+00  8.96093711e+00
  6.52310263e-06]
obj =  16.337089478959697 , x =  [-4.094048   11.94308    -9.97792738  5.00956087 -7.34826061]
obj =  12.65171859658793 , x =  [12.          0.95855872  3.9998168  15.16535124  3.92859738]
obj =  20.149187975830483 , x =  [12.         13.07472027 -8.90644254 15.65517763  7.62950117]
obj =  -4.133372662587448 , x =  [ 1.04814621e+00 -1.04814732e+00  1.04814770e+00  1.81085640e-07
 -5.02104432e-07]
obj =  7.887351129729055 , x =  [ 6.00349467  8.01823317 -4.99958323  6.98855627  5.00967699]
obj =  13.76818243657544 , x =  [11.76477036 -2.89266184 -7.99227508 -0.02113335  7.00176788]
obj =  12.628854933326224 , x =  [10.85568971 -0.94247395 -3.99889807  2.17098564 -4.89992576]


e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `fin

In [10]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
false_count = (df['success'] == False).sum()
print(f"Count of rows where 'success' is False: {false_count}")
df_filtered = df[df['success'] != False]
df_filtered.describe()

NaN count in column 'obj': 0
Count of rows where 'success' is False: 2


,obj,x_1,x_2,x_3,x_4,x_5,initial_x1,initial_x2,initial_x3,initial_x4,initial_x5,nfev
count,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000
mean,10.816701,4.217013,1.708534,-1.659435,5.598960,1.244335,3.600177,-1.128011,0.338439,3.820924,1.616538,167.459184
std,4.002290,4.863888,7.900772,5.615330,5.681776,5.082519,4.491347,12.504815,5.705403,9.294824,9.134453,53.521008
min,-4.133373,-4.993261,-7.998511,-10.000000,-4.038105,-7.953395,-4.537560,-19.895738,-9.739653,-11.850767,-14.889930,86.000000
25%,8.914919,0.015031,-5.704153,-6.090855,1.101755,-2.705616,0.369042,-11.496278,-5.007472,-3.496350,-5.929091,131.500000
50%,10.655789,4.528365,-0.430618,-2.073824,4.991826,1.031121,3.901508,-1.783072,0.911391,2.609188,1.501329,161.500000
75%,13.587662,8.000320,8.642046,3.942858,9.014447,3.991224,6.945312,10.977965,4.914774,11.676205,8.360708,189.500000
max,21.069282,12.000000,20.991196,8.999256,18.176799,13.783562,11.397368,21.180373,9.777242,19.838408,17.950608,340.000000


### PSO

In [59]:
re_run_PSO = True
re_run_save_PSO = True
sol = 'PSO'
case_name = 'Case3T'
runs_number = 100

In [56]:
from pyswarm import pso

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
obj_list = []
iterations=[]
state_list = []
itera = 0
def func(x):
    global itera
    obj, c1, c2, c3, c4 = Case3T(x)
    cons = [c1+1,2-c1,c2+1,4-c2,c3+2,3-c3,c4,1.5-c4]
    global x_p
    try:
        equal = np.array_equal(x,x_p)
        if not equal:
            itera=itera+1
    except:
        itera=itera+1
    x_p =x
    return obj,cons

if re_run_PSO == True or re_run_save_PSO == True:
    for num in range(runs_number):
        xopt, fopt = pso(lambda x:func(x)[0], lb =[-5, -20,-10,-2,-15] ,ub=[12,22,10,20,18],f_ieqcons=lambda x:func(x)[1:2], swarmsize=100, omega=0.5, phip=0.5, phig=0.5, maxiter=200, minstep=1e-8,minfunc=1e-8, debug=False)
        state = np.all(np.array(func(xopt)[1:])>= 0)
        itera=itera-1         
        print( 'fopt =', fopt, 'xopt = ', xopt, 'state = ', state) 
        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        obj_list.append(fopt)
        x_1_list.append(xopt[0])
        x_2_list.append(xopt[1])
        x_3_list.append(xopt[2])
        x_4_list.append(xopt[3])
        x_5_list.append(xopt[4])
        state_list.append(state)
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,    
        'iterations': iterations,
        'state':state_list
        })
        if re_run_save_PSO == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)

Stopping search: maximum iterations reached --> 200
However, the optimization couldn't find a feasible design. Sorry
fopt = 1e+100 xopt =  [-2.24014126 -7.71348842 -0.2044162  14.01850528 -5.89084359] state =  False
Stopping search: Swarm best position change less than 1e-08
fopt = 18.646454915354507 xopt =  [ 6.31752731  9.31678863 -6.02779974  2.17248434  0.43066603] state =  True
Stopping search: Swarm best position change less than 1e-08
fopt = 16.632630214331744 xopt =  [ 0.23675839  1.2049993   7.85835107 11.97743212 -1.16380771] state =  True
Stopping search: Swarm best objective change less than 1e-08
fopt = 17.59369354235447 xopt =  [ 1.19068317e-01  2.15984144e-03  4.32574498e+00  9.42350608e+00
 -4.30124658e+00] state =  True
Stopping search: maximum iterations reached --> 200
However, the optimization couldn't find a feasible design. Sorry
fopt = 1e+100 xopt =  [  0.26392216  -2.33288284  -9.70192596  19.67465893 -13.7856714 ] state =  False
Stopping search: Swarm best posi

In [61]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
false_count = (df['obj'] == 1e+100).sum()
print(f"Count of rows where 'obj' is 1e+100: {false_count}")
df_filtered = df[df['obj'] != 1e+100]
df_filtered = df_filtered[df_filtered['state'] == True]
df_filtered.describe()

NaN count in column 'obj': 0
Count of rows where 'obj' is 1e+100: 18


,obj,x_1,x_2,x_3,x_4,x_5,iterations
count,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000
mean,16.913035,3.568219,0.678391,0.717473,7.585518,0.874312,11112.756098
std,2.806661,3.970055,6.148974,4.606619,4.546567,5.035304,5484.052304
min,7.559195,-3.929551,-7.859630,-10.000000,-1.396457,-7.837577,3738.000000
25%,16.006522,0.304620,-4.442999,-2.524550,4.804260,-2.768651,6168.250000
50%,17.633536,3.288805,-0.420922,1.556836,7.471546,0.148257,10694.000000
75%,18.892300,6.854085,5.783874,4.112256,10.580302,5.475540,15330.250000
max,20.598417,10.242863,12.027420,9.133701,17.237864,11.136327,20100.000000


In [51]:
for i in df_filtered.values:
    if np.all(np.array(func(i[1:6])[1:])<= 0) == False:
        print(False)

### GA

In [53]:
re_run_GA = True
re_run_save_GA = True
sol = 'GA'
case_name = 'Case3T'
runs_number = 100

In [43]:
from sko.GA import GA
from sko.operators import crossover

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
obj_list = []
iterations = []
state_list = []
itera = 0
def func(x):
    global itera
    obj, c1, c2, c3, c4 = Case3T(x)
    cx1 = -c1-1
    cx11=c1-2
    cx2=-1-c2
    cx22=c2-4
    cx3=-2-c3
    cx33=c3-3
    cx4=-c4
    cx44=c4-1.5
    itera=itera+1
    return obj,cx1,cx2,cx3,cx4,cx11,cx22,cx33,cx44

if re_run_GA == True or re_run_save_GA == True:
    for num in range(runs_number):
        ga = GA(func=lambda x:func(x)[0], n_dim=5, size_pop=100, max_iter=200,lb =[-5, -20,-10,-2,-15] ,ub=[12,22,10,20,18], precision=1e-7,prob_mut = 0.001, constraint_ueq=[lambda x:func(x)[i] for i in range(1,9)])
        ga.register(operator_name='crossover', operator=crossover.crossover_2point_prob, crossover_prob=0.5)
        best_x, best_y = ga.run()
        state = np.all(np.array(func(best_x)[1:])<= 0)
        itera=itera-1 
        print( 'obj =', best_y, 'x = ', best_x,'state = ', state)

        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        
        obj_list.append(best_y[0])
        x_1_list.append(best_x[0])
        x_2_list.append(best_x[1])
        x_3_list.append(best_x[2])
        x_4_list.append(best_x[3])
        x_5_list.append(best_x[4])
        state_list.append(state)

        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list, 
        'iterations': iterations,
        'state':state_list
        })
        if re_run_save_GA == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)

obj = [-1.98276258] x =  [-0.99777823  0.99777757 -0.9977809   1.99555408 -1.99555397] state =  False
obj = [-4.05093216] x =  [ 8.82558528e-05 -1.99233502e+00  1.19581819e-05 -9.96157359e-01
  1.14161521e-05] state =  False
obj = [-4.54644217] x =  [-9.95568331e-01 -1.46776438e-06 -9.95559510e-01  9.95558720e-01
 -9.95549632e-01] state =  False
obj = [3.45916972] x =  [-9.99441780e-01 -5.99660527e+00 -2.79396773e-06  9.99461811e-01
 -3.99773636e+00] state =  False
obj = [0.38152198] x =  [ 2.99778706e+00 -2.99638170e+00  1.99753170e+00  9.98825338e-01
 -5.15878201e-05] state =  False
obj = [-1.25209826] x =  [ 0.99814973 -1.99626097 -0.99807736  1.99628776 -1.9962735 ] state =  False
obj = [-1.72424297] x =  [ 7.33882191e-07 -1.99583273e+00  1.99583252e+00  1.99583298e+00
  6.59376383e-07] state =  False
obj = [-1.48130837] x =  [ 1.99607721e+00 -1.99607579e+00 -1.99607481e+00 -9.98036291e-01
 -7.54371287e-07] state =  False
obj = [5.3063228] x =  [ 1.99935174e+00 -8.99708105e+00 -5.6

In [54]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")

df_filtered = df[df['state'] == True]
df_filtered.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,x_3,x_4,x_5,iterations
count,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.0
mean,0.807609,1.997761,0.998876,-1.997754,0.99888,-3.995508,180001.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.807609,1.997761,0.998876,-1.997754,0.99888,-3.995508,180001.0
25%,0.807609,1.997761,0.998876,-1.997754,0.99888,-3.995508,180001.0
50%,0.807609,1.997761,0.998876,-1.997754,0.99888,-3.995508,180001.0
75%,0.807609,1.997761,0.998876,-1.997754,0.99888,-3.995508,180001.0
max,0.807609,1.997761,0.998876,-1.997754,0.99888,-3.995508,180001.0
